Use vietocr conda env for this.

# Define data/result directory

In [15]:
import os

data_folder = "../data"
result_folder = "../results"
manga_list = "vi/Ruri Dragon (Oneshot)/Ch. None"  # Will be changed to list later

# Ruri Dragon
manga_folder = os.path.join(data_folder, manga_list)

individual_result_folder = os.path.join(result_folder, manga_list)
json_output_dir = os.path.join(individual_result_folder, "json_results")
result_image_output_dir = os.path.join(individual_result_folder, "image_results")

cut_texts_dir = os.path.join(individual_result_folder, "cut_texts")

raw_images = os.listdir(manga_folder)
json_files = os.listdir(json_output_dir)
easy_ocr_result_dir = os.path.join(individual_result_folder, "easy_ocr_result")
os.makedirs(easy_ocr_result_dir, exist_ok=True)

# Initialize model

In [ ]:
import easyocr

# Initialize EasyOCR (Vietnamese)
reader = easyocr.Reader(["vi"])

d:\miniconda3\envs\vietocr\lib\site-packages\torch\nn\modules\transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


# Predict

In [ ]:
import os
import json
from PIL import Image

for json_file in json_files:
    if not json_file.endswith(".json"):
        continue

    base_name = os.path.splitext(json_file)[0]
    json_path = os.path.join(json_output_dir, json_file)
    cut_page_dir = os.path.join(cut_texts_dir, base_name)

    # skip if cut_texts folder doesn't exist
    if not os.path.exists(cut_page_dir):
        print(f"⚠️ No cut_texts found for {json_file}, skipping...")
        continue

    # load json
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    new_ocr = []
    for idx, (bbox, is_essential) in enumerate(
        zip(data["texts"], data["is_essential_text"])
    ):
        if not is_essential:
            # keep placeholder (so indexes align)
            new_ocr.append(data["ocr"][idx])
            continue

        cut_img_path = os.path.join(cut_page_dir, f"{base_name}_{idx:03}.png")
        if not os.path.exists(cut_img_path):
            print(f"⚠️ Missing cut image {cut_img_path}, keeping original OCR.")
            new_ocr.append(data["ocr"][idx])
            continue

        img = Image.open(cut_img_path).convert("RGB")  # EasyOCR works better in RGB
        try:
            results = reader.readtext(
                cut_img_path, detail=0, paragraph=True
            )  # returns only text
            vi_text = results[0] if results else data["ocr"][idx]  # fallback if empty
        except Exception as e:
            print(f"❌ OCR failed for {cut_img_path}: {e}")
            vi_text = data["ocr"][idx]  # fallback to original
        new_ocr.append(vi_text)

    # replace OCR field
    data["ocr"] = new_ocr

    # save to easy_ocr_result
    out_path = os.path.join(easy_ocr_result_dir, json_file)
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

    print(f"✅ Saved EasyOCR result for {json_file} → {out_path}")

⚠️ No cut_texts found for 00.json, skipping...
✅ Saved EasyOCR result for 01.json → ../results\vi/Ruri Dragon (Oneshot)/Ch. None\easy_ocr_viet_ocr_result\01.json
✅ Saved EasyOCR result for 02.json → ../results\vi/Ruri Dragon (Oneshot)/Ch. None\easy_ocr_viet_ocr_result\02.json
✅ Saved EasyOCR result for 03.json → ../results\vi/Ruri Dragon (Oneshot)/Ch. None\easy_ocr_viet_ocr_result\03.json
✅ Saved EasyOCR result for 04.json → ../results\vi/Ruri Dragon (Oneshot)/Ch. None\easy_ocr_viet_ocr_result\04.json
✅ Saved EasyOCR result for 05.json → ../results\vi/Ruri Dragon (Oneshot)/Ch. None\easy_ocr_viet_ocr_result\05.json
✅ Saved EasyOCR result for 06.json → ../results\vi/Ruri Dragon (Oneshot)/Ch. None\easy_ocr_viet_ocr_result\06.json
✅ Saved EasyOCR result for 07.json → ../results\vi/Ruri Dragon (Oneshot)/Ch. None\easy_ocr_viet_ocr_result\07.json
✅ Saved EasyOCR result for 08.json → ../results\vi/Ruri Dragon (Oneshot)/Ch. None\easy_ocr_viet_ocr_result\08.json
✅ Saved EasyOCR result for 09.jso

Individual test

In [17]:
# OCR a text box with multiple lines
from PIL import Image

test_image_dir = "../data/vi/test_data/multiple_line"
test_images = os.listdir(test_image_dir)
for test_image in test_images:
    img = os.path.join(test_image_dir, test_image)
    result = reader.readtext(img, detail=0, paragraph=True)
    print(f"{test_image}: {result}")

03_006.png: ['Ù, TRÔNG CON GIỖNG HỆT BỐ CON ĐẪY.']
04_001.png: ['VÌ CON LÀ Con Lai GlỮA NGƯỜI VÀ RỒNG.']
05_004.png: ['BINH TINH NÃO, ĐÂU PHẢl NÓ SẼ GlẾT CON HAY Gi ĐÂU.']


In [19]:
# OCR a text box with one lines
from PIL import Image

test_image_dir = "../data/vi/test_data/one_line"
test_images = os.listdir(test_image_dir)
for test_image in test_images:
    img = os.path.join(test_image_dir, test_image)
    result = reader.readtext(img, detail=0, paragraph=True)
    print(f"{test_image}: {result}")

04_001.png: ['VÌ COn LÀ']
05_004.png: ['BINH TINH']
